# A simple model
copied from ptrblck https://discuss.pytorch.org/t/cnn-always-outputs-the-same-values-whatever-the-input-image/110853/8

In [2]:
import torch
import torch.nn.functional as F
import torch.nn as nn

In [8]:
class ConvNet(nn.Module):
    def __init__(self):
        super(ConvNet, self).__init__()
        self.conv1 = nn.Conv2d(3, 32, kernel_size=5)
        self.conv2 = nn.Conv2d(32,64, kernel_size=5)
        self.conv3 = nn.Conv2d(64,128, kernel_size=5)
        self.pool = nn.MaxPool2d(3,3)
        self.fc1 = nn.Linear(128*5*5, 512)
        self.fc2 = nn.Linear(512, 256)
        self.fc3 = nn.Linear(256,1)
        self.check = []
    
    def forward(self, x):        
        x = self.pool(F.relu(self.conv1(x)))        
        x = self.pool(F.relu(self.conv2(x)))  
        self.check.append(x)
        if len(self.check) > 1:
            # print((self.check[-1] == self.check[-2]).float().sum())
            print(self.check[-1].size(0))
        x = self.pool(F.relu(self.conv3(x)))       
        x = x.view(x.size(0), -1)
        x = F.relu(self.fc1(x))        
        x = F.relu(self.fc2(x))        
        x = self.fc3(x)
        return x

model = ConvNet()
data = torch.randn(10, 3, 200, 200)
target = torch.randint(0, 2, (10, 1)).float()
criterion = nn.BCEWithLogitsLoss()    
optimizer = torch.optim.Adam(model.parameters(), lr=1e-3)

for epoch in range(100):
    optimizer.zero_grad()    
    outputs = model(data)
    loss = criterion(outputs, target)
    loss.backward()
    optimizer.step()
    print('epoch {}, loss {}'.format(epoch, loss.item()))

acc = ((outputs > 0.0) == target).float().mean()
print(acc)

epoch 0, loss 0.7182826399803162
10
epoch 1, loss 0.5158636569976807
10
epoch 2, loss 1.4494097232818604
10
epoch 3, loss 0.5360206961631775
10
epoch 4, loss 0.5953057408332825
10
epoch 5, loss 0.6537932753562927
10
epoch 6, loss 0.6647077798843384
10
epoch 7, loss 0.6634024381637573
10
epoch 8, loss 0.6539944410324097
10
epoch 9, loss 0.6335905194282532
10
epoch 10, loss 0.5959726572036743
10
epoch 11, loss 0.5398167371749878
10
epoch 12, loss 0.4997752606868744
10
epoch 13, loss 0.5485011339187622
10
epoch 14, loss 0.5604231357574463
10
epoch 15, loss 0.5183058977127075
10
epoch 16, loss 0.49838900566101074
10
epoch 17, loss 0.508897602558136
10
epoch 18, loss 0.5226853489875793
10
epoch 19, loss 0.5263373255729675
10
epoch 20, loss 0.5196897387504578
10
epoch 21, loss 0.5074461698532104
10
epoch 22, loss 0.49816447496414185
10
epoch 23, loss 0.4997873306274414
10
epoch 24, loss 0.5090368390083313
10
epoch 25, loss 0.5119657516479492
10
epoch 26, loss 0.5045756101608276
10
epoch 27, 

# averaging 2 models
ptrblck: you could get the state_dicts of both models, average the parameters and reload the new state_dict.
Here is a small dummy example:
https://discuss.pytorch.org/t/average-each-weight-of-two-models/77008

In [3]:
# Setup
modelA = nn.Linear(1, 1)
modelB = nn.Linear(1, 1)

sdA = modelA.state_dict()
sdB = modelB.state_dict()

# Average all parameters
for key in sdA:
    sdB[key] = (sdB[key] + sdA[key]) / 2.

# Recreate model and load averaged state_dict (or use modelA/B)
model = nn.Linear(1, 1)
model.load_state_dict(sdB)

<All keys matched successfully>

# Ensembel model
https://discuss.pytorch.org/t/combining-trained-models-in-pytorch/28383/2
 use a models output (modelA, modelB) in the forward function of the zusammengesetzte model MyEnsemble

In [ ]:
class MyModelA(nn.Module):
    def __init__(self):
        super(MyModelA, self).__init__()
        self.fc1 = nn.Linear(10, 2)
        
    def forward(self, x):
        x = self.fc1(x)
        return x
    

class MyModelB(nn.Module):
    def __init__(self):
        super(MyModelB, self).__init__()
        self.fc1 = nn.Linear(20, 2)
        
    def forward(self, x):
        x = self.fc1(x)
        return x


class MyEnsemble(nn.Module):
    def __init__(self, modelA, modelB):
        super(MyEnsemble, self).__init__()
        self.modelA = modelA
        self.modelB = modelB
        self.classifier = nn.Linear(4, 2)
        
    def forward(self, x1, x2):
        x1 = self.modelA(x1)
        x2 = self.modelB(x2)
        x = torch.cat((x1, x2), dim=1)
        x = self.classifier(F.relu(x))
        return x

# Create models and load state_dicts    
modelA = MyModelA()
modelB = MyModelB()
# Load state dicts
modelA.load_state_dict(torch.load(PATH))
modelB.load_state_dict(torch.load(PATH))

model = MyEnsemble(modelA, modelB)
x1, x2 = torch.randn(1, 10), torch.randn(1, 20)
output = model(x1, x2)